### Part 1.9 - Extract_data_45000_movies

#### Import the libraries

In [ ]:
# For cleaning and preparing the dataset
# -> dataframe manipulation
# -> text manipulation
# -> Web Scrapping

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import os

# Module to serialize the content produced from the execution of the code

import pickle

# Module to monitor the progress of a python for loop

from tqdm import tqdm_notebook
# Example of Use: tqdm_notebook(examples, desc="Converting examples to features")

#### Ninth 5000 movies (45,000)

Extract data about:

* field 1: Plot Summary
* field 2: Actors
* field 3: Directors
* field 4: IMDB rating
* field 5: Plot Synopsis
* field 6: Reviews

In [ ]:
"""
Open the movie_content urls for the ninth 5000 movies and create the souplist of those URLs
"""
content_nine = []

with open(os.path.join(os.getcwd(), 'movie_content_url\\data_nine_14012020.pkl'), 'rb') as f:
    
    content_nine = pickle.load(f)

print("Number of URLs: {}".format(len(content_nine)))

content_souplist_nine = []

for i in tqdm_notebook(content_nine):
    
    content_souplist_nine.append(BeautifulSoup(i.text))

print("Number of souplists: {}".format(len(content_souplist_nine)))

In [ ]:
"""
Open the synopsis urls of the ninth 5000 movies and create the souplist of those URLs
"""
synopsis_nine = []

with open(os.path.join(os.getcwd(), 'synopsis_url\\synopsis_nine_14012020.pkl'), 'rb') as f:
    
    synopsis_nine = pickle.load(f)

print("Number of URLs: {}".format(len(synopsis_nine)))

synopsis_souplist_nine = []

for i in tqdm_notebook(synopsis_nine):
    
    synopsis_souplist_nine.append(BeautifulSoup(i.text))

print("Number of souplists: {}".format(len(synopsis_souplist_nine)))

In [ ]:
"""
Open the reviews urls of the ninth 5000 movies and create the souplist of those URLs
"""
review_nine = []

with open(os.path.join(os.getcwd(), 'reviews_url\\review_nine_16012020.pkl'), 'rb') as f:
    
    review_nine = pickle.load(f)

print("Number of URLs: {}".format(len(review_nine)))

review_souplist_nine = []

for i in tqdm_notebook(review_nine):
    
    review_souplist_nine.append(BeautifulSoup(i.text))

print("Number of review tags: {}".format(len(review_souplist_nine)))

In [ ]:
"""
Field 1: Extract plot summary
"""
myfield_plot = []
plot_summary = []
index_to_remove_no_plot = []

[myfield_plot.append(i.find_all('div', {'class':'plot_summary'})) for i in tqdm_notebook(content_souplist_nine)]

[[[plot_summary.append(y.text) for y in x.find_all('div', {'class':'summary_text'})] for x in i] if len(i) !=0 else index_to_remove_no_plot.append(myfield_plot.index(i)) for i in myfield_plot]
      
print("Length of Plot Summary list: {}".format(len(plot_summary)))
print("Length of the list with Movies that don't have plot summary: {}".format(len(index_to_remove_no_plot)))
if len(index_to_remove_no_plot) == 0:
    print("None of the movie miss plot")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_plot))

In [ ]:
"""
Field 2: Extract actors
"""
myfield_cast = []
phase_two = []
phase_three = []
actors_list = []
index_to_remove_no_actors = []

[myfield_cast.append(i.find_all('table', {'class':'cast_list'})) for i in tqdm_notebook(content_souplist_nine)]
    
r_one = re.compile(".*name")

[[phase_two.append(j.find_all('a', {'href':r_one})) for j in i] for i in myfield_cast]

[phase_three.append(phase_two[i][1::2]) for i in range(len(phase_two))]

[actors_list.append(list(map(lambda x: x.text.strip(' ').replace('\n', ''), actors))) for actors in phase_three]            
    
index_to_remove_no_actors = [i for i,x in enumerate(myfield_cast) if not x]
        
print("Length of Actors list: {}".format(len(actors_list)))
print("Length of the list with Movies that don't have actors: {}".format(len(index_to_remove_no_actors)))
if len(index_to_remove_no_actors) == 0:
    print("None of the movie miss actors")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_actors))

In [ ]:
"""
Field 3: Extract director name(s)
"""
myfield_director = []
director_name = []
index_to_remove_no_directors = []

[myfield_director.append(i.find_all('div', {'class':'plot_summary'})) for i in tqdm_notebook(content_souplist_nine)]

r_name = re.compile(".*name")

[[director_name.append(j.find_all('a', {'href':r_name})) for j in i] for i in myfield_director]
    
director_names = [item[0].text for item in director_name if len(item)!=0]

index_to_remove_no_directors = [i for i,x in enumerate(myfield_director) if not x]

print("Length of Directors list: {}".format(len(director_names)))
print("Length of the list with Movies that don't have directors: {}".format(len(index_to_remove_no_directors)))
if len(index_to_remove_no_directors) == 0:
    print("None of the movie miss directors")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_directors))

In [ ]:
"""
Field 4: Extract imdb movie rating
"""
myfield_rating = []
ratings = []
index_to_remove_no_rating = []

[myfield_rating.append(i.find_all('div', {'class':'ratingValue'})) for i in tqdm_notebook(content_souplist_nine)]

[[[ratings.append(y.text) for y in x.find_all('span', {'itemprop':'ratingValue'})] for x in i] for i in myfield_rating]

index_to_remove_no_rating = [i for i,x in enumerate(myfield_rating) if not x]

print("Length of Ratings list: {}".format(len(ratings)))
print("Length of the list with Movies that are not rated: {}".format(len(index_to_remove_no_rating)))
if len(index_to_remove_no_rating) == 0:
    print("None of the movie miss ratings")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_rating))

In [ ]:
"""
Field 5: Extract plot synopsis

Having extracted the synopsis content for each movie, we noticed that roughly more than 50% percent of the movies have a synopsis text.
Thus, in order not to erase almost half of the first 5,000 we will not further proceed in keeping the synopsis text
"""
synopsis_step_one = []
synopsis_step_two = []
synopsis_step_three = []

[synopsis_step_one.append(i.find_all('ul', {'class':'ipl-zebra-list', 'id':'plot-synopsis-content'})) for i in tqdm_notebook(synopsis_souplist_nine)]

[[synopsis_step_two.append(j.find_all('li', {'class':'ipl-zebra-list__item'})) for j in i] for i in synopsis_step_one]

[[synopsis_step_three.append(j.text.strip(' ').replace('\n', '').replace('\\', '')) for j in i] for i in synopsis_step_two]

index_to_remove_no_synopsis = [i for i,x in enumerate(synopsis_step_one) if not x]

print("Length of Synopsis list: {}".format(len(synopsis_step_three)))
print("Length of the list with Movies that don't have synopsis: {}".format(len(index_to_remove_no_synopsis)))
if len(index_to_remove_no_synopsis) == 0:
    print("None of the movies miss a synopsis")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_synopsis))

In [ ]:
"""
Print the movies with no synopsis text.
Result: We observed that many movies missed a synopsis text. Thus, we decided to not use synopsis for furhter analysis (although will be extracted)
"""
print([i for i,x in enumerate(synopsis_step_one) if not x])
print([i for i,x in enumerate(synopsis_step_two) if not x])
print([i for i,x in enumerate(synopsis_step_three) if not x])

# Many of the movies do not have a synopsis text, this is it won't be used for furhter analysis (althpugh will be extracted)

In [ ]:
"""
Field 6: Extract movie reviews
"""
myfield_review_step_one = []
myfield_review_step_two = []
myfield_review_step_three = []

[myfield_review_step_one.append(i.find_all('div', {'class':'lister-list'})) for i in tqdm_notebook(review_souplist_nine)]

[[myfield_review_step_two.append(j.find_all('div', {'class':'text show-more__control'})) for j in i] for i in myfield_review_step_one]

[myfield_review_step_three.append(list(map(lambda x: x.text, reviews))) for reviews in myfield_review_step_two]

index_to_remove_no_review = [i for i,x in enumerate(myfield_review_step_one) if not x]

print("Length of Reviews list: {}".format(len(myfield_review_step_three)))
print("Length of the list with Movies that don't have reviews: {}".format(len(index_to_remove_no_review)))
if len(index_to_remove_no_review) == 0:
    print("None of the movies miss reviews")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_review))

In [ ]:
"""
Print the movies with no user reviews text.
"""
print([i for i,x in enumerate(myfield_review_step_one) if not x])
print([i for i,x in enumerate(myfield_review_step_two) if not x])
print([i for i,x in enumerate(myfield_review_step_three) if not x])

<b> - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - </b>

#### Dataframe creation based on the movie content, synopsis and reviews extracted

In [ ]:
"""
Import the movie_content (the extracted HTML document of the column imdb_url)
Import the reviews content (the extracted HTML document of the column reviews_url)
"""
with open(os.path.join(os.getcwd(), 'movie_content_url\\data_nine_14012020.pkl'), 'rb') as f:
    
    content_nine = pickle.load(f)

print("Number of URLs: {}".format(len(content_nine)))

with open(os.path.join(os.getcwd(), 'reviews_url\\review_nine_16012020.pkl'), 'rb') as f:
    
    review_nine = pickle.load(f)

print("Number of URLs: {}".format(len(review_nine)))

In [ ]:
"""
Below we import the 5 fields extracted previously
However, we we imported the "rating" file, we noticed that they did not downloaded properly. Actually, the respective list was empty,
thus we decided to download again the data in order no to make any other mistake with the index of the movies.
"""
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_nine\\pre-indexed files\\plot_nine_18012020.pkl'), 'rb') as f:
    
    plot = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_nine\\pre-indexed files\\rating_nine_18012020.pkl'), 'rb') as f:
    
    rating = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_nine\\pre-indexed files\\actors_nine_18012020.pkl'), 'rb') as f:
    
    actors = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_nine\\pre-indexed files\\director_nine_18012020.pkl'), 'rb') as f:
    
    director = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_nine\\pre-indexed files\\synopsis_nine_18012020.pkl'), 'rb') as f:
    
    synopsis = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_nine\\pre-indexed files\\reviews_nine_18012020.pkl'), 'rb') as f:
    
    reviews = pickle.load(f)
    
print(len(plot))
print(len(rating))
print(len(actors))
print(len(director))
print(len(synopsis))
print(len(reviews))

print(rating) #empty

In [ ]:
"""
Running the commands below we can spot the number of indexes that should be removed from the data extracted.
"Add a Plot": Denoted that the movies does have a written plot on its IMDB website.
"It looks like we don\'t have a Synopsis for this title yet. Be the first to contribute! Just click the "Edit page" button at the bottom of the page or learn more in the Synopsis submission guide.": Also denotes a non-written synopsis.

Synopsis indexes are not further considered. There are too many movies with no synopsis text and there is no poin in erasing almost half of the first 5000 movies due to missing synopsis text.
"""
matching_add_plot = [s for s, x in enumerate(plot) if "Add a Plot" in x]

print("Number of movies with 'Add Plot': ", len(matching_add_plot), '\n')

matching_add_synopsis = [s for s, x in enumerate(synopsis) if 'It looks like we don\'t have a Synopsis for this title yet. Be the first to contribute! Just click the "Edit page" button at the bottom of the page or learn more in the Synopsis submission guide.' in x]

print("Number of movies with no synopsis text: ", len(matching_add_synopsis))

In [ ]:
"""
Remove the faulty indexes from the data. As faulty is characterized an index with no data relevant to movie content or user review of a movie or with an error 404 web page.
Indexes collected during the data extraction of the 6 fields and from the above python cell that prints number of movies with no written plot.
"""
index_remove=[26,30,53,71,99,181,203,238,262,264,308,311,326,327,377,446,497,521,554,665,667,668,669,676,719,733,736,737,738,751,757,758,762,765,767,768,770,771,772,773,774,775,776,777,779,780,782,783,839,840,894,903,953,959,962,1037,1044,1057,1216,1242,1355,1360,1436,1437,1438,1439,1444,1445,1448,1449,1454,1491,1592,1609,1610,1647,1691,1787,1790,1859,1875,1886,1887,1993,2013,2090,2094,2102,2116,2120,2123,2128,2406,2437,2450,2554,2558,2559,2577,2584,2590,2593,2611,2613,2711,2765,2836,3052,3211,3228,3299,3300,3425,3534,3780,3781,4021,4153,4258,4507,4549,4577,4583,4586,4617,4724,4725,4733,4786,4951,4971,4990,4991,1028,1790,2043,3497,4491,4536,4562,4910,17,18,38,45,55,63,68,83,92,125,130,142,147,154,186,193,202,206,210,241,242,251,255,267,313,328,377,386,401,402,453,455,468,469,473,485,529,535,537,549,554,569,598,609,626,629,642,645,660,661,667,670,676,678,686,709,722,724,734,754,757,765,771,773,778,793,812,833,838,871,906,907,908,910,911,926,998,1028,1147,1160,1270,1278,1368,1525,1650,1674,1717,1842,1845,1859,1886,1887,1901,1931,1942,1993,1999,2051,2055,2056,2066,2091,2115,2117,2119,2120,2167,2297,2358,2386,2417,2420,2569,2575,2577,2629,2943,2966,3034,3108,3187,3199,3202,3204,3300,3315,3316,3331,3335,3337,3349,3351,3529,3531,3594,3635,3636,3702,4032,4208,4223,4224,4227,4231,4262,4264,4265,4266,4288,4304,4415,4470,4516,4572,4576,4581,4595,4707,4764,4767,4945,4972,4975]
index_remove = list(dict.fromkeys(index_remove))
print(len(index_remove))
content_index_nine = [i for j, i in enumerate(content_nine) if j not in index_remove]
review_index_nine = [i for j, i in enumerate(review_nine) if j not in index_remove]
print(len(content_index_nine))
print(len(review_index_nine))

In [ ]:
"""
Run this cell once! For every re-execution of this notebook, please use the already pickled file.
"""
# with open(os.path.join(os.getcwd(), '58,000 movies\\movie_nine\\content_index_nine_20012020.pkl'), 'wb') as f:
#     pickle.dump(content_index_nine, f)
    
# with open(os.path.join(os.getcwd(), '58,000 movies\\movie_nine\\review_index_nine_20012020.pkl'), 'wb') as f:
#     pickle.dump(review_index_nine, f)

In [ ]:
"""
Import the pruned list of movie_content & reviews web pages. The indexes that have been removed did not contain one or more -
of the following fields: plot, actors, director(s), imdb_rating, review(s)

It is important to re-run the process of extracting the information in order to assure that we have extracted the correct information and text for each movie
Otherwise, we might observe mismatches in the fields assigned to each movie title.
"""
content_souplist_nine = []

with open(os.path.join(os.getcwd(), '58,000 movies\\movies_nine\\content_index_nine_20012020.pkl'), 'rb') as f:
    
    content_index_nine = pickle.load(f)

print("Number of URLs: {}".format(len(content_index_nine)))

for i in tqdm_notebook(content_index_nine):
    
    content_souplist_nine.append(BeautifulSoup(i.text))

print("Number of souplists: {}".format(len(content_souplist_nine)))

#-----------------------------------------------------------------------------------------------------

with open(os.path.join(os.getcwd(), '58,000 movies\\movies_nine\\review_index_nine_20012020.pkl'), 'rb') as f:
    
    review_nine = pickle.load(f)

print("Number of URLs: {}".format(len(review_nine)))

review_souplist_nine = []

for i in tqdm_notebook(review_nine):
    
    review_souplist_nine.append(BeautifulSoup(i.text))

print("Number of review tags: {}".format(len(review_souplist_nine)))

In [ ]:
"""
Field 1: Extract plot summary
"""
myfield_plot = []
plot_summary = []
index_to_remove_no_plot = []

[myfield_plot.append(i.find_all('div', {'class':'plot_summary'})) for i in tqdm_notebook(content_souplist_nine)]

[[[plot_summary.append(y.text) for y in x.find_all('div', {'class':'summary_text'})] for x in i] if len(i) !=0 else index_to_remove_no_plot.append(myfield_plot.index(i)) for i in myfield_plot]
      
print("Length of Plot Summary list: {}".format(len(plot_summary)))
print("Length of the list with Movies that don't have plot summary: {}".format(len(index_to_remove_no_plot)))
if len(index_to_remove_no_plot) == 0:
    print("None of the movie miss plot")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_plot))

In [ ]:
"""
Field 2: Extract actors
"""
myfield_cast = []
phase_two = []
phase_three = []
actors_list = []
index_to_remove_no_actors = []

[myfield_cast.append(i.find_all('table', {'class':'cast_list'})) for i in tqdm_notebook(content_souplist_nine)]
    
r_one = re.compile(".*name")

[[phase_two.append(j.find_all('a', {'href':r_one})) for j in i] for i in myfield_cast]

[phase_three.append(phase_two[i][1::2]) for i in range(len(phase_two))]

[actors_list.append(list(map(lambda x: x.text.strip(' ').replace('\n', ''), actors))) for actors in phase_three]            
    
index_to_remove_no_actors = [i for i,x in enumerate(myfield_cast) if not x]
        
print("Length of Actors list: {}".format(len(actors_list)))
print("Length of the list with Movies that don't have actors: {}".format(len(index_to_remove_no_actors)))
if len(index_to_remove_no_actors) == 0:
    print("None of the movie miss actors")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_actors))

In [ ]:
"""
Field 3: Extract director name(s)
"""
myfield_director = []
director_name = []
index_to_remove_no_directors = []

[myfield_director.append(i.find_all('div', {'class':'plot_summary'})) for i in tqdm_notebook(content_souplist_nine)]

r_name = re.compile(".*name")

[[director_name.append(j.find_all('a', {'href':r_name})) for j in i] for i in myfield_director]
    
director_names = [item[0].text for item in director_name]

index_to_remove_no_directors = [i for i,x in enumerate(myfield_director) if not x]

print("Length of Directors list: {}".format(len(director_names)))
print("Length of the list with Movies that don't have directors: {}".format(len(index_to_remove_no_directors)))
if len(index_to_remove_no_directors) == 0:
    print("None of the movie miss directors")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_directors))

In [ ]:
"""
Field 4: Extract imdb movie rating
"""
myfield_rating = []
ratings = []
index_to_remove_no_rating = []

[myfield_rating.append(i.find_all('div', {'class':'ratingValue'})) for i in tqdm_notebook(content_souplist_nine)]

[[[ratings.append(y.text) for y in x.find_all('span', {'itemprop':'ratingValue'})] for x in i] for i in myfield_rating]

index_to_remove_no_rating = [i for i,x in enumerate(myfield_rating) if not x]

print("Length of Ratings list: {}".format(len(ratings)))
print("Length of the list with Movies that are not rated: {}".format(len(index_to_remove_no_rating)))
if len(index_to_remove_no_rating) == 0:
    print("None of the movie miss ratings")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_rating))

In [ ]:
"""
Field 5: Extract movie user reviews
"""
myfield_review_step_one = []
myfield_review_step_two = []
myfield_review_step_three = []

[myfield_review_step_one.append(i.find_all('div', {'class':'lister-list'})) for i in tqdm_notebook(review_souplist_nine)]

[[myfield_review_step_two.append(j.find_all('div', {'class':'text show-more__control'})) for j in i] for i in myfield_review_step_one]

[myfield_review_step_three.append(list(map(lambda x: x.text, reviews))) for reviews in myfield_review_step_two]

index_to_remove_no_review = [i for i,x in enumerate(myfield_review_step_one) if not x]

print("Length of Reviews list: {}".format(len(myfield_review_step_three)))
print("Length of the list with Movies that don't have reviews: {}".format(len(index_to_remove_no_review)))
if len(index_to_remove_no_review) == 0:
    print("None of the movies miss reviews")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_review))

In [ ]:
print(len([i for i,x in enumerate(myfield_review_step_one) if not x]))

print([i for i,x in enumerate(myfield_review_step_two) if not x])

print([i for i,x in enumerate(myfield_review_step_three) if not x])

In [ ]:
"""
Run this cell once! For every re-execution of this notebook, please use the already pickled file.
"""
# # Pickle the requests file for further use!

# with open(os.path.join(os.getcwd(), '58,000 movies\\movies_nine\\plot_nine_20012020.pkl'), 'wb') as f:
#     pickle.dump(plot_summary, f)
    
# with open(os.path.join(os.getcwd(), '58,000 movies\\movies_nine\\actors_nine_20012020.pkl'), 'wb') as f:
#     pickle.dump(actors_list, f)
    
# with open(os.path.join(os.getcwd(), '58,000 movies\\movies_nine\\director_nine_20012020.pkl'), 'wb') as f:
#     pickle.dump(director_names, f)

# with open(os.path.join(os.getcwd(), '58,000 movies\\movies_nine\\reviews_nine_24012020.pkl'), 'wb') as f:
#     pickle.dump(myfield_review_step_nine, f)

In [ ]:
# with open(os.path.join(os.getcwd(), '58,000 movies\\movies_nine\\rating_nine_20012020.pkl'), 'wb') as f:
#     pickle.dump(ratings, f)

In [ ]:
"""
Remove the faulty indexes from the data. As faulty is characterized an index with no data relevant to movie content or user review of a movie or with an error 404 web page.
Indexes collected during the data extraction of the 6 fields and from the above python cell that prints number of movies with no written plot.
"""
index_remove=[26,30,53,71,99,181,203,238,262,264,308,311,326,327,377,446,497,521,554,665,667,668,669,676,719,733,736,737,738,751,757,758,762,765,767,768,770,771,772,773,774,775,776,777,779,780,782,783,839,840,894,903,953,959,962,1037,1044,1057,1216,1242,1355,1360,1436,1437,1438,1439,1444,1445,1448,1449,1454,1491,1592,1609,1610,1647,1691,1787,1790,1859,1875,1886,1887,1993,2013,2090,2094,2102,2116,2120,2123,2128,2406,2437,2450,2554,2558,2559,2577,2584,2590,2593,2611,2613,2711,2765,2836,3052,3211,3228,3299,3300,3425,3534,3780,3781,4021,4153,4258,4507,4549,4577,4583,4586,4617,4724,4725,4733,4786,4951,4971,4990,4991,1028,1790,2043,3497,4491,4536,4562,4910,17,18,38,45,55,63,68,83,92,125,130,142,147,154,186,193,202,206,210,241,242,251,255,267,313,328,377,386,401,402,453,455,468,469,473,485,529,535,537,549,554,569,598,609,626,629,642,645,660,661,667,670,676,678,686,709,722,724,734,754,757,765,771,773,778,793,812,833,838,871,906,907,908,910,911,926,998,1028,1147,1160,1270,1278,1368,1525,1650,1674,1717,1842,1845,1859,1886,1887,1901,1931,1942,1993,1999,2051,2055,2056,2066,2091,2115,2117,2119,2120,2167,2297,2358,2386,2417,2420,2569,2575,2577,2629,2943,2966,3034,3108,3187,3199,3202,3204,3300,3315,3316,3331,3335,3337,3349,3351,3529,3531,3594,3635,3636,3702,4032,4208,4223,4224,4227,4231,4262,4264,4265,4266,4288,4304,4415,4470,4516,4572,4576,4581,4595,4707,4764,4767,4945,4972,4975]
index_remove = list(dict.fromkeys(index_remove))
dataset = pd.read_pickle(os.path.join(os.getcwd(), "dataset_58,000_14012020_latest_version.pkl"))
dataset_nine = dataset.iloc[40000:45000].reset_index(drop=True)
dataset_nine = dataset_nine[~dataset_nine.index.isin(index_remove)]
dataset_nine.shape

In [ ]:
"""
Loading the data extracted.
Those are the most important data source across all the notebook. 
The whole data extraction part was executed to end up with the following 6 files per mini-batch.

The examiner can directly import the files below to review the movie content extracted for the nineth mini-batch.
"""
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_nine\\plot_nine_20012020.pkl'), 'rb') as f:
    
    plot = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_nine\\rating_nine_20012020.pkl'), 'rb') as f:
    
    rating = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_nine\\actors_nine_20012020.pkl'), 'rb') as f:
    
    actors = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_nine\\director_nine_20012020.pkl'), 'rb') as f:
    
    director = pickle.load(f)
        
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_nine\\reviews_nine_24012020.pkl'), 'rb') as f:
    
    reviews = pickle.load(f)
    
assert len(plot) == len(rating) == len(actors) == len(director) == len(reviews)

In [ ]:
dataset_nine['actors'] = actors
dataset_nine['plot'] = plot
dataset_nine['imdb_rating'] = rating
dataset_nine['director'] = director
dataset_nine['reviews'] = reviews
dataset_nine = dataset_nine.drop(["movieId", "imdbId", "synopsis_url"], axis=1)

In [ ]:
index_remove=[]
index_remove=[2,10,21,28,33,41,42,77,81,93,95,97,99,100,102,127,142,166,186,209,214,256,257,260,270,282,312,319,335,337,340,344,363,374,375,376,379,381,382,395,403,404,420,436,438,443,453,455,469,476,478,480,486,493,503,506,511,530,545,560,573,575,584,597,599,601,603,642,643,649,654,655,659,664,666,667,691,701,707,725,728,739,745,763,787,791,797,798,819,849,859,872,895,921,922,928,935,940,941,946,950,959,965,969,971,981,983,985,986,988,1011,1018,1022,1023,1025,1026,1031,1070,1074,1078,1082,1088,1091,1092,1110,1118,1143,1164,1165,1207,1211,1214,1225,1229,1236,1238,1306,1321,1362,1391,1418,1429,1493,1505,1517,1524,1538,1564,1565,1603,1616,1617,1632,1637,1650,1651,1680,1681,1682,1687,1690,1717,1718,1723,1725,1728,1734,1755,1780,1799,1800,1810,1814,1816,1825,1826,1831,1833,1834,1844,1862,1868,1878,1879,1880,1881,1893,1903,1907,1911,1914,1919,1953,1959,1970,1993,2008,2009,2024,2053,2081,2089,2106,2119,2154,2155,2159,2160,2163,2188,2215,2216,2226,2248,2309,2329,2349,2350,2382,2413,2415,2417,2418,2419,2424,2426,2429,2430,2435,2436,2437,2446,2454,2457,2468,2474,2475,2483,2484,2493,2506,2539,2574,2628,2630,2638,2678,2689,2695,2696,2697,2711,2716,2720,2730,2734,2737,2750,2762,2777,2779,2781,2782,2784,2805,2872,2877,2881,2887,2919,2965,2972,2984,3012,3034,3060,3065,3067,3077,3086,3092,3095,3099,3100,3103,3104,3107,3108,3112,3120,3121,3141,3144,3148,3149,3150,3151,3155,3168,3171,3195,3202,3206,3211,3221,3241,3293,3359,3360,3394,3409,3419,3434,3469,3489,3490,3491,3494,3495,3499,3512,3564,3566,3570,3608,3610,3611,3613,3614,3619,3642,3648,3661,3662,3666,3670,3677,3700,3701,3708,3712,3732,3752,3753,3759,3770,3771,3835,3882,3911,3916,3917,3922,3950,3951,3963,3969,3974,3980,3982,3992,3993,3995,3997,4009,4010,4011,4012,4016,4030,4031,4036,4039,4043,4068,4088,4090,4119,4127,4162,4202,4206,4209,4211,4250,4261,4274,4280,4287,4295,4314,4317,4323,4333,4348,4350,4364,4383,4388,4426,4456,4460,4461,4471,4475,4476,4479,4482,4483,4490,4491,4497,4500,4534,4550,4551,4560,4561,4564,4572,4586,4602,4617,4622,4627,4632,4634,4642,4651,4663,4673,4675,4681,4688,4689,4690,4704,4710]
dataset_nine = dataset_nine[~dataset_nine.index.isin(index_remove)]
dataset_nine = dataset_nine[dataset_nine.astype(str)['reviews'] != '[]']
dataset_nine.shape

In [ ]:
"""
This is the table to use in Part 1, and the table that will be concatenated with the rest of the mini-batch tables.
"""
# dataset_nine.to_pickle(os.path.join(os.getcwd(), "58,000 movies\\movie_nine\\dataset_nine_final_25012020.pkl"))